In [31]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pickle
import datetime

## 1.Load Data

In [32]:
import nltk
nltk.download('reuters')
nltk.download('punkt')  # For tokenization


[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
from nltk.corpus import reuters as rt 
rt.words()

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', ...]

In [34]:
categories = rt.categories() 

print(categories)

['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr', 'dmk', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-oil', 'heat', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-oil', 'livestock', 'lumber', 'meal-feed', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-oil', 'palmkernel', 'pet-chem', 'platinum', 'potato', 'propane', 'rand', 'rape-oil', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'rye', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']


In [35]:
#. tokenization
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt_tab')

# Get raw text from the reuters corpus
raw_text = rt.raw(categories=['fuel', 'gas','gold', 'grain', 'heat', 'housing', 'income', 'interest']) 

# Tokenize into sentences and then words
sentences = sent_tokenize(raw_text)  # Tokenize the raw text into sentences
corpus = [word_tokenize(sent.lower()) for sent in sentences]  # Tokenize sentences into words

# Check the number of tokenized sentences
print(f"Number of tokenized sentences: {len(corpus)}")


print(corpus[0])


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Number of tokenized sentences: 8458
['china', 'daily', 'says', 'vermin', 'eat', '7-12', 'pct', 'grain', 'stocks', 'a', 'survey', 'of', '19', 'provinces', 'and', 'seven', 'cities', 'showed', 'vermin', 'consume', 'between', 'seven', 'and', '12', 'pct', 'of', 'china', "'s", 'grain', 'stocks', ',', 'the', 'china', 'daily', 'said', '.']


In [36]:
len(corpus)

8458

In [37]:
#2. numeralizaition
# find uniquee words
flatten=lambda l: [item for sublist in l for item in sublist]
#assign unique integer
vocabs=list(set(flatten(corpus))) # all the words we have in the system


In [38]:
# create hand mapping between iteger and word
word2index={v:idx for idx, v in enumerate(vocabs)}


In [39]:
lst_idx=len(vocabs)

In [40]:
lst_idx

14269

In [41]:
#append UNK for unknown
vocabs.append('<UNK>')

In [42]:
word2index['<UNK>']= lst_idx

In [43]:
len(vocabs)

14270

In [44]:
index2word={v:k for k, v in word2index.items()}

## 2. Prepare train data

In [45]:
# create pairs of center word, and outsidee word
def random_batch(batch_size,corpus):
    skipgrams=[]
    # loop  each corpus
    for doc in corpus:
        #look from the 3rd word until third last word since window size =2
        for i in range(2, len(doc)-2):
            #center word
            center=word2index[doc[i]]
            #outside words=2 words
            outside = (word2index[doc[i-2]],word2index[doc[i-1]], 
                       word2index[doc[i+1]],word2index[doc[i+2]])
            #print(center, outside)
            #for each for these two outside words, we gonna append to a list
            for each_out in outside:
                #print(each_out)
                skipgrams.append([center,each_out])
            # center, outeside1; center, outside2

    random_index=np.random.choice(range(len(skipgrams)),batch_size,replace=False)
    input, label=[], []
    for index in random_index:
        input.append([skipgrams[index][0]])
        label.append([skipgrams[index][1]])
    return np.array(input), np.array(label)

x, y=random_batch(2,corpus)

In [46]:
x

array([[4724],
       [5769]])

In [47]:
y

array([[1269],
       [3746]])

## 3.Model

### Word2Vec(Skipgram)
$$J(\theta) = -\frac{1}{T}\sum_{t=1}^{T}\sum_{\substack{-m \leq j \leq m \\ j \neq 0}}\log P(w_{t+j} | w_t; \theta)$$

where $P(w_{t+j} | w_t; \theta) = $

$$P(o|c)=\frac{\exp(\mathbf{u_o^{\top}v_c})}{\sum_{w=1}^V\exp(\mathbf{u_w^{\top}v_c})}$$

where $o$ is the outside words and $c$ is the center word

In [48]:
voc_size   = len(vocabs)
emb_size = 2

In [49]:
# create skipgram model
class Skipgram(nn.Module):
    def __init__(self, voc_size, emb_size):
        super(Skipgram,self).__init__()
        self.embedding_center=nn.Embedding(voc_size,emb_size)
        self.embedding_outside=nn.Embedding(voc_size,emb_size)
    def forward(self,center,outside,all_vocabs):
        center_embedding=self.embedding_center(center) #(batch_size, 1, emb_size)
        outside_embedding=self.embedding_center(outside) #(batch_size, 1, emb_size)
        all_vocabs_embedding=self.embedding_center(all_vocabs) #(batch_size, voc_size, emb_size

        top_term=torch.exp(outside_embedding.bmm(center_embedding.transpose(1,2)).squeeze(2))  # bmm is dot product (ignore batch size) and reduce dim to 2
        #batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) = (batch_size, 1)
        
        lower_term=all_vocabs_embedding.bmm(center_embedding.transpose(1,2)).squeeze(2)
        #batch_size, voc_size, emb_size) @ (batch_size, emb_size, 1) = (batch_size, voc_size, 1) = (batch_size, voc_size) 
        
        lower_term_sum=torch.sum(torch.exp(lower_term),1) #(batch_size,1)
        
        #calculate loss
        loss=-torch.mean(torch.log(top_term/lower_term_sum))
        
        return loss
    def get_vector(self, word):
        id_tensor = torch.LongTensor([word2index[word]])
        id_tensor = id_tensor
        v_embed = self.embedding_v(id_tensor)
        u_embed = self.embedding_u(id_tensor) 
        word_embed = (v_embed + u_embed) / 2 
        # x, y = word_embed[0][0].item(), word_embed[0][1].item()

        return word_embed


In [50]:
# prepare all vocabs
batch_size=2
voc_size= len(vocabs)
def prepare_sequence(seq, word2index):
    idxs=list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"],seq))
    return  torch.LongTensor(idxs)
all_vocabs=prepare_sequence(list(vocabs),word2index).expand(batch_size,voc_size)
all_vocabs

tensor([[    0,     1,     2,  ..., 14267, 14268, 14269],
        [    0,     1,     2,  ..., 14267, 14268, 14269]])

In [51]:
model_skipgram=Skipgram(voc_size,2)

In [52]:
x_skipgram, y_skipgram = random_batch(batch_size, corpus)
x_tensor_skipgram = torch.LongTensor(x_skipgram)
y_tensor_skipgram = torch.LongTensor(y_skipgram)

In [53]:
loss_skipgram = model_skipgram(x_tensor_skipgram, y_tensor_skipgram, all_vocabs)
loss_skipgram

tensor(10.5296, grad_fn=<NegBackward0>)

## 4. Training

In [54]:
batch_size     = 2 # mini-batch size
model_skipgram      = Skipgram(voc_size, emb_size)
optimizer_skipgram  = optim.Adam(model_skipgram.parameters(), lr=0.001)

In [55]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    mins = elapsed_time // 60
    secs = elapsed_time % 60
    return int(mins), int(secs)


In [56]:
# Training
import time
num_epochs = 1000
start = time.time()

for epoch in range(num_epochs):
    
    #get batch
    input_batch_skipgram, label_batch_skipgram = random_batch(batch_size, corpus)
    input_tensor_skipgram = torch.LongTensor(input_batch_skipgram)
    label_tensor_skipgram = torch.LongTensor(label_batch_skipgram)
     
    #predict
    loss_skipgram = model_skipgram(input_tensor_skipgram, label_tensor_skipgram, all_vocabs)
    
    #backprogate
    optimizer_skipgram.zero_grad()
    loss_skipgram.backward()
    
    #update alpha
    optimizer_skipgram.step()
    
    #print the loss
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1} | Loss: {loss_skipgram:2.6f}") #Epoch 6 front space, 0 back space

end = time.time()
epoch_mins, epoch_secs = epoch_time(start, end)

print(f"time: {epoch_mins}m {epoch_secs}s")

Epoch 100 | Loss: 10.956932
Epoch 200 | Loss: 9.925303
Epoch 300 | Loss: 8.831079
Epoch 400 | Loss: 10.877717
Epoch 500 | Loss: 9.682178
Epoch 600 | Loss: 10.013674
Epoch 700 | Loss: 9.777369
Epoch 800 | Loss: 14.556937
Epoch 900 | Loss: 10.726488
Epoch 1000 | Loss: 9.682384
time: 9m 42s


In [57]:
print(f'Training Loss: {loss_skipgram}, Training Time: {epoch_mins}m {epoch_secs}s')

Training Loss: 9.682384490966797, Training Time: 9m 42s


In [62]:
# # Saving the model for testing
torch.save(model_skipgram.state_dict(), 'D:/AIT_lecture/NLP/code\Assignment/NLP-2025/NLP-A1/models/skipgram_v1.pt')

In [59]:
# Data = {
#     'corpus': corpus,
#     'vocab': vocabs,
#     'word2index': word2index,
#     'voc_size': voc_size,
#     'embedding_size': emb_size
# }

In [60]:
# pickle.dump(Data,open('./models/Data.pkl', 'wb'))